https://www.tensorflow.org/recommenders/examples/quickstart

In [1]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

c:\Users\luisc\miniconda3\envs\tfrs\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Ratings data
ratings = tfds.load('movielens/100k-ratings', split='train')

# Features of all the available movies
movies = tfds.load('movielens/100k-movies', split='train')

# Select the basic features
ratings = ratings.map(
    lambda x: {
        'movie_title': x['movie_title'],
        'user_id': x['user_id']
    }
)

movies = movies.map(lambda x: x['movie_title'])

Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...: 100%|██████████| 1/1 [00:03<00:00,  3.37s/ url]

Dl Completed...: 100%|██████████| 1/1 [00:03<00:00,  3.62s/ url]


Dataset movielens downloaded and prepared to ~\tensorflow_datasets\movielens\100k-ratings\0.1.1. Subsequent calls will reuse this data.


Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...: 100%|██████████| 1/1 [00:00<00:00, 124.99 url/s]
Extraction completed...: 0 file [00:00, ? file/s]
Dl Completed...: 100%|██████████| 1/1 [00:00<00:00, 62.49 url/s] 


Dataset movielens downloaded and prepared to ~\tensorflow_datasets\movielens\100k-movies\0.1.1. Subsequent calls will reuse this data.


In [3]:
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x['user_id']))

movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

In [4]:
class MovieLensModel(tfrs.Model):
    # We derive from a custom base class to help reduce bolierplate. Under the hood, thesee are astill plain keras model

    def __init__(
        self,
        user_model: tf.keras.Model,
        movie_model: tf.keras.Model,
        task: tfrs.tasks.Retrieval
    ):
        super().__init__()

        # Set up user and movie representations
        self.user_model = user_model
        self.movie_model = movie_model

        # Set up a retrieval task
        self.task = task

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        # Define how the loss is computed
        user_embeddings = self.user_model(features['user_id'])
        movie_embeddings = self.movie_model(features['movie_title'])

        return self.task(user_embeddings, movie_embeddings)

In [5]:
# Define user and movie models
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 64)
])

movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocab_size(), 64)
])

# Define your objectives
task = tfrs.tasks.Retrieval(
    metrics=tfrs.metrics.FactorizedTopK(
        movies.batch(128).map(movie_model)
    )
)

In [6]:
# Create a retrieval model
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))
model.fit(ratings.batch(4096), epochs=3)

# Use brute-force search to set up retrieval using the trained representations
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(movies.batch(100).map(lambda title: (title, model.movie_model(title))))

# Get some recommendations
_, titles = index(np.array(['42']))
print(f'Top 3 recommendations for user 42: {titles[0, :3]}')

Epoch 1/3
25/25 [==============================] - 9s 198ms/step - factorized_top_k/top_1_categorical_accuracy: 8.0000e-05 - factorized_top_k/top_5_categorical_accuracy: 0.0016 - factorized_top_k/top_10_categorical_accuracy: 0.0052 - factorized_top_k/top_50_categorical_accuracy: 0.0438 - factorized_top_k/top_100_categorical_accuracy: 0.1006 - loss: 33081.1203 - regularization_loss: 0.0000e+00 - total_loss: 33081.1203
Epoch 2/3
25/25 [==============================] - 5s 190ms/step - factorized_top_k/top_1_categorical_accuracy: 1.4000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0051 - factorized_top_k/top_10_categorical_accuracy: 0.0138 - factorized_top_k/top_50_categorical_accuracy: 0.1049 - factorized_top_k/top_100_categorical_accuracy: 0.2107 - loss: 31010.8409 - regularization_loss: 0.0000e+00 - total_loss: 31010.8409
Epoch 3/3
25/25 [==============================] - 4s 178ms/step - factorized_top_k/top_1_categorical_accuracy: 3.9000e-04 - factorized_top_k/top_5_categorica